# Estimating losses with the Backward Monte Carlo scheme

This example shows how to run BMC (Backward Monte Carlo) simulations.

BMC is a program that can be thought as running ASCOT5 backwards in time.

In [ ]:
import numpy as np
import unyt
import matplotlib.pyplot as plt

from a5py import Ascot

a5 = Ascot("ascot.h5", create=True)

a5.data.create_input("bfield analytical iter circular")
a5.data.create_input("plasma flat")
a5.data.create_input("E_TC")
a5.data.create_input("N0_1D")
a5.data.create_input("Boozer")
a5.data.create_input("MHD_STAT")
a5.data.create_input("asigma_loc")

Wall

In [ ]:
w2d = {"nelements":8,
       "r":np.array([2.3, 2.3, -2.1, -2.1,  2.3,  2.3,  1.9, 1.9]) + 6.0,
       "z":np.array([0.1, 2.1,  2.1, -2.1, -2.1, -0.1, -0.1, 0.1]),
       "flag":np.array([[0,0,0,0,0,0,0,1]]).T}
a5.data.create_input("wall_2D", **w2d)

plt.plot(w2d["r"], w2d["z"])
plt.show()

Options

In [ ]:
from a5py.ascot5io.options import Opt

opt = Opt.get_default()
opt.update({
    #
    "BMC_TIMEDEPENDENT":0,
    #
    "BMC_ORBIT_SUBCYCLES":100,
    #
    "BMC_TIMESTEP":1.0e-05, "BMC_TSTART":0, "BMC_TSTOP":1e-3,
    #"BMC_TIMESTEP":1.0e-06, "BMC_TSTART":0, "BMC_TSTOP":1.1e-6,
    #
    "BMC_MASS":4.007, "BMC_CHARGE":2, "BMC_ANUM":4, "BMC_ZNUM":2,
    "ENABLE_COULOMB_COLLISIONS":1, "DISABLE_ENERGY_CCOLL":1,
    # Distribution output
    "ENABLE_DIST_5D":1,
    "DIST_MIN_R":3.8,  "DIST_MAX_R":8.5, "DIST_NBIN_R":800,
    "DIST_MIN_Z":-2.2, "DIST_MAX_Z":2.2, "DIST_NBIN_Z":800,
    "DIST_MIN_PHI":0,        "DIST_MAX_PHI":360,     "DIST_NBIN_PHI":1,
    #"DIST_MIN_PPA":-0.1e-19, "DIST_MAX_PPA":0.1e-19, "DIST_NBIN_PPA":50,
    #"DIST_MIN_PPE":0,        "DIST_MAX_PPE":0.1e-19, "DIST_NBIN_PPE":50,
    "DIST_MIN_PPA":1.0e4*1.6e-19, "DIST_MAX_PPA":2e4*1.6e-19, "DIST_NBIN_PPA":3,
    "DIST_MIN_PPE":-1.0,     "DIST_MAX_PPE":1.0, "DIST_NBIN_PPE":50,
    "DIST_MIN_TIME":0,       "DIST_MAX_TIME":1.0,    "DIST_NBIN_TIME":1,
    "DIST_MIN_CHARGE":1.5,   "DIST_MAX_CHARGE":2.5,  "DIST_NBIN_CHARGE":1,
})
a5.data.create_input("opt", **opt)

Run

In [ ]:
import subprocess
subprocess.run(["./../../build/bmc_main", "--d=\"BMC\""])
#print("Simulation completed")

Read output

In [ ]:
a5 = Ascot("ascot.h5")
dist = a5.data.BMC.getdist("5d")
dist.integrate(charge=np.s_[:], time=np.s_[:], phi=np.s_[:])

dist1 = dist.integrate(ppar=np.s_[:], pperp=np.s_[:], copy=True)
dist1._distribution = dist1.histogram()

dist2 = dist.integrate(r=np.s_[:], z=np.s_[:], copy=True)
dist2._distribution = dist2.histogram()

fig = plt.figure(figsize=(7,5))
ax1 = fig.add_subplot(1,2,1)
ax2 = fig.add_subplot(1,2,2)

dist1.plot(axes=ax1)
dist2.plot(axes=ax2)

idx = np.arange(w2d["r"].size+1)
idx[-1] = 0
ax1.plot(w2d["r"][idx], w2d["z"][idx], color="white")
# ax2.set_aspect("equal", adjustable="box")
# plt.colorbar(h, orientation="horizontal")
a5.input_init(bfield=True)
a5.input_plotrhocontour(axes=ax1)
a5.input_free()
plt.show()

Interpolate and create markers

In [ ]:
from a5py.ascot5io.marker import Marker
from a5py import physlib

dist = a5.data.BMC.getdist("5d")

r = 4.9*unyt.m
z = 0.0*unyt.m
#ppar  = -0.2e-19*unyt.kg*unyt.m/unyt.s
#pperp =  0.2e-19*unyt.kg*unyt.m/unyt.s
ppar  =  1e4*1.6e-19*unyt.kg*unyt.m/unyt.s
pperp =  -0.5*unyt.kg*unyt.m/unyt.s

dist.integrate(charge=np.s_[:], time=np.s_[:], phi=np.s_[:])
dist._distribution = dist.histogram()
print(dist.interpolate(r=r, z=z, ppar=ppar, pperp=pperp))

mrk = Marker.generate("gc", n=100, species="alpha")

pnorm = np.sqrt(ppar**2 + pperp**2)
ekin  = physlib.energy_momentum(mrk["mass"][0], pnorm)
pitch = ppar / pnorm

mrk["energy"][:] = ekin
mrk["pitch"][:]  = pitch
mrk["r"][:]      = r
mrk["z"][:]      = z

a5.data.create_input("gc", desc="FMC", activate=True, **mrk)

Set simulation options and run

In [ ]:
opt = Opt.get_default()
opt.update({
    # Simulation mode
    "SIM_MODE":2, "ENABLE_ADAPTIVE":1,
    # Setting max mileage above slowing-down time is a good safeguard to ensure
    # simulation finishes even with faulty inputs. Same with the CPU time limit.
    "ENDCOND_SIMTIMELIM":1, "ENDCOND_MAX_MILEAGE":1e-3, "ENDCOND_WALLHIT":1,
    # Physics
    "ENABLE_ORBIT_FOLLOWING":1, "ENABLE_COULOMB_COLLISIONS":1,
    "DISABLE_ENERGY_CCOLL":1,
})
a5.data.create_input("opt", desc="FMC", activate=True, **opt)

In [ ]:
import subprocess
subprocess.run(["./../../build/ascot5_main", "--d=\"FMC\""])
print("Simulation completed")

Compare loss fraction to FILD to the BMC value

In [ ]:
a5 = Ascot("ascot.h5")
#a5.data.active.getstate("ekin", state="ini").to("MeV")
a5.data.active.getstate("walltile", state="end")

One can use BMC result for importance sampling.